In [1]:
# This Note book will train a 4-hidden layer ANN on Mnist data set
# with tensorflow


import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets('/tmp/data/')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
mnist

In [3]:
n_inputs=28*28
n_hidden1=300
n_hidden2=200
n_hidden3=100
n_hidden4=50
n_output=10

In [4]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
Y=tf.placeholder(tf.int64,shape=(None),name='Y')

In [5]:
def layer(x,n_neurons,name,activation=None):
    with tf.name_scope(name):
        n_inputs=int(x.get_shape()[1])
        stddev=2/np.sqrt(n_inputs)
        init=tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)
        w=tf.Variable(init, name='weights')
        bias=tf.Variable(tf.zeros((n_neurons)))
        z=tf.matmul(x,w)+bias
        if activation=='relu':
            return tf.nn.relu(z)
        else:
            return z
        

In [6]:
with tf.name_scope('Dnn'):
    hidden1=layer(X,n_hidden1,'hidden_layer1',activation='relu')
    hidden2=layer(hidden1,n_hidden2,'hidden_layer2',activation='relu')
    hidden3=layer(hidden2,n_hidden3,'hidden_layer3',activation='relu')
    hidden4=layer(hidden3,n_hidden4,'hidden_layer4',activation='relu')
    logits=layer(hidden4,n_output,'outputs')


In [7]:
with tf.name_scope('loss'):
    entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,logits=logits)
    loss=tf.reduce_mean(entropy,name='loss')

In [8]:
learning_rate=0.01
epsilon=0.1
with tf.name_scope('train'):
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate,epsilon=epsilon)
    train_op=optimizer.minimize(loss)

In [9]:
with tf.name_scope('eval'):
    correct=tf.nn.in_top_k(logits,Y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [10]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [11]:
from datetime import datetime

now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir='Mnist'
logdir='{}/run-{}/'.format(root_logdir,now)

In [12]:
loss_summary=tf.summary.scalar('Loss',loss)
acc_train_summary=tf.summary.scalar('train_acc',accuracy)
acc_test_summary=tf.summary.scalar('test_acc',accuracy)
file_writer=tf.summary.FileWriter(logdir, tf.get_default_graph())

In [13]:
n_epochs=400
batch_size=50

In [14]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            x_batch, y_batch=mnist.train.next_batch(batch_size)
            sess.run(train_op,feed_dict={X:x_batch,Y:y_batch})
        summary_str_loss=loss_summary.eval(feed_dict={X:x_batch,Y:y_batch})
        summary_str_train=acc_train_summary.eval(feed_dict={X:x_batch,Y:y_batch})
        summary_str_test=acc_test_summary.eval(feed_dict={X:mnist.test.images,Y:mnist.test.labels})
        file_writer.add_summary(summary_str_loss,epoch)
        file_writer.add_summary(summary_str_train,epoch)
        file_writer.add_summary(summary_str_test,epoch)
    file_writer.close()
            

In [ ]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            x_batch, y_batch=mnist.train.next_batch(batch_size)
            sess.run(train_op,feed_dict={X:x_batch,Y:y_batch})
            lo=loss.eval(feed_dict={X:x_batch, Y:y_batch})
            acc_train=accuracy.eval(feed_dict={X:x_batch, Y:y_batch})
            acc_test=accuracy.eval(feed_dict={X:mnist.test.images,Y:mnist.test.labels})
        print(epoch,'Train accuracy:',acc_train,'Test accuracy:',acc_test,'Loss:',lo)